In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import time
import copy
import pickle
start_time = time.time()


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import re
pattern_category='\s*(A[0-9]..*)'
pattern_firstRow='([a-zA-Z0-9]+\S)\s+([0-9]+)\s+(.*\S)\s\s+([0-9]+\s*.\s*[0-9]+)'
pattern_note = '^(n|N)(o|O)(t|T)(e|E)' #'^[nN][oO][tT][eE]'
pattern_edited='^\s+EDITED UNIVERSE:(.*)'
pattern_valid='\s*VALID ENTR[YIES]+'
#pattern_valid_details = '\s+([-0-9]+)\s+(.*\S)'
pattern_appendingLine='\s*(.*)\s+' #appending info



In [ ]:
import pandas as pd
file=open("/content/drive/My Drive/BOC/2021CPSRules_update.txt",'r')
line = file.readline()
while not line.strip().startswith('NAME'):
  line = file.readline()

data_output=[]
item_num=0
mark='NOTE'
category=''
name=''
size='' 
description=''
location="'"
edited_universe=''
valid_entry=''
note=''
category_transit=''
for line in file:
  if line.strip()=='End of Basic \ Portion of the Record':
    break
  if not line.split():
      mark='NOTE' 
      continue
    
  if re.match(pattern_category, line):
    category_new = line.strip()
    if not category:
      category=category_new
    elif not category_new == category:
      category_transit=category_new
    
  elif re.match(pattern_firstRow, line):
    item_num+=1
    if item_num>=2:
        data_output.append([item_num-1,category,name, size, description, location, edited_universe, valid_entry, note])
        name=''
        size='' 
        description=''
        location=''
        edited_universe=''
        valid_entry=''
        note=''

    if category_new==category_transit:
        category=category_new
    mark='DE'
    matchObj=re.match(pattern_firstRow, line)
    name=name+matchObj.group(1).strip()
    size=size+matchObj.group(2).strip()
    description=description+matchObj.group(3).strip()
    location=location+matchObj.group(4).strip()

  elif re.match(pattern_note, line):
    mark='NOTE'
    note=note+line.strip()
  elif re.match(pattern_edited, line):
    mark='ED'
    matchObj=re.match(pattern_edited, line)
    edited_universe=edited_universe+matchObj.group(1).strip()
  elif re.match(pattern_valid, line):
      mark='VA'
      line=file.readline()
      if not line.split():
        line=file.readline()
      valid_entry = valid_entry + ' ' + line.strip()
  elif mark=='NOTE':
    note = note+'  '+line.strip()
  elif mark=='DE':
    description = description+'  '+line.strip()
  elif mark=='ED':
    edited_universe = edited_universe+'  '+line.strip()
  elif mark=='VA':
    line=line.strip()
    if re.match('^[0-9]+.[0-9]*.*',line):
      valid_entry = valid_entry +' @_rule_@ '+line
    else:
      valid_entry = valid_entry +' '+line

data_df = pd.DataFrame(data_output)
data_df.columns=['item_num', 'category', 'name', 'size', 'description', 'location', 'edited_universe', 'valid_entry','note']

#data_df.iloc[200:202]
data_df.head()

,item_num,category,name,size,description,location,edited_universe,valid_entry,note
0,1,A1. HOUSEHOLD INFORMATION,HRHHID,15,HOUSEHOLD IDENTIFIER (Part 1),'1-15,ALL HHLD's IN SAMPLE,,Part 1. See Characters 71-75 for Part 2 of the Household Identifier.
1,2,A1. HOUSEHOLD INFORMATION,HRMONTH,2,MONTH OF INTERVIEW,16-17,ALL HHLDs IN SAMPLE,01 MIN VALUE @_rule_@ 12 MAX VALUE,
2,3,A1. HOUSEHOLD INFORMATION,HRYEAR4,4,YEAR OF INTERVIEW,18-21,ALL HHLDs IN SAMPLE,1998 MIN VALUE @_rule_@ 2999 MAX VALUE,
3,4,A1. HOUSEHOLD INFORMATION,HURESPLI,2,LINE NUMBER OF THE CURRENT RESPONDENT,22-23,,0 MIN VALUE @_rule_@ 99 MAX VALUE,
4,5,A1. HOUSEHOLD INFORMATION,HUFINAL,3,FINAL OUTCOME CODE,24-26,,001 FULLY COMPLETE CATI INTERVIEW @_rule_@ 002 PARTIALLY COMPLETED CATI INTERVIEW @_rule_@ 003 ...,OUTCOME CODES BETWEEN 001 AND 020 ARE FOR CATI. ALL OTHER OUTCOME CODES ARE FOR CAPI.


In [ ]:
data_df_output=copy.deepcopy(data_df)
data_df_output.insert(0,'year', '2021')
data_df_output['valid_entry']=data_df_output['valid_entry'].str.replace(' @_rule_@ ', '\n')

In [ ]:
data_df_output.to_csv('/content/drive/My Drive/BOC/dictionary_table.csv', encoding='utf-8-sig')

In [ ]:
pd.set_option('max_colwidth',None)
data_df[data_df['name']=='HUFINAL']

,item_num,category,name,size,description,location,edited_universe,valid_entry,note
4,5,A1. HOUSEHOLD INFORMATION,HUFINAL,3,FINAL OUTCOME CODE,24-26,,"001 FULLY COMPLETE CATI INTERVIEW @_rule_@ 002 PARTIALLY COMPLETED CATI INTERVIEW @_rule_@ 003 COMPLETE BUT PERSONAL VISIT REQUESTED NEXT MONTH @_rule_@ 004 PARTIAL, NOT COMPLETE AT CLOSEOUT @_rule_@ 005 LABOR FORCE COMPLETE, SUPPLEMENT INCOMPLETE - CATI @_rule_@ 006 LF COMPLETE, SUPPLEMENT DK ITEMS INCOMPLETE AT CLOSEOUT–ASEC ONLY @_rule_@ 020 HH OCCUPIED ENTIRELY BY ARMED FORCES MEMBERS OR ALL UNDER 15 YEARS OF AGE @_rule_@ 201 CAPI COMPLETE @_rule_@ 202 CALLBACK NEEDED @_rule_@ 203 SUFFICIENT PARTIAL - PRECLOSEOUT @_rule_@ 204 SUFFICIENT PARTIAL - AT CLOSEOUT @_rule_@ 205 LABOR FORCE COMPLETE, - SUPPL. INCOMPLETE - CAPI @_rule_@ 213 LANGUAGE BARRIER @_rule_@ 214 UNABLE TO LOCATE @_rule_@ 216 NO ONE HOME @_rule_@ 217 TEMPORARILY ABSENT @_rule_@ 218 REFUSED @_rule_@ 219 OTHER OCCUPIED - SPECIFY @_rule_@ 223 ENTIRE HOUSEHOLD ARMED FORCES @_rule_@ 224 ENTIRE HOUSEHOLD UNDER 15 @_rule_@ 225 TEMP. OCCUPIED W/PERSONS WITH URE @_rule_@ 226 VACANT REGULAR @_rule_@ 227 VACANT - STORAGE OF HHLD FURNITURE @_rule_@ 228 UNFIT, TO BE DEMOLISHED @_rule_@ 229 UNDER CONSTRUCTION, NOT READY @_rule_@ 230 CONVERTED TO TEMP BUSINESS OR STORAGE @_rule_@ 231 UNOCCUPIED TENT OR TRAILER SITE @_rule_@ 232 PERMIT GRANTED - CONSTRUCTION NOT STARTED @_rule_@ 233 OTHER - SPECIFY @_rule_@ 240 DEMOLISHED @_rule_@ 241 HOUSE OR TRAILER MOVED @_rule_@ 242 OUTSIDE SEGMENT @_rule_@ 243 CONVERTED TO PERM. BUSINESS OR STORAGE @_rule_@ 244 MERGED @_rule_@ 245 CONDEMNED @_rule_@ 247 UNUSED SERIAL NUMBER OR LISTING SHEET @_rule_@ 248 OTHER - SPECIFY @_rule_@ 258 UNLOCATABLE SAMPLE ADDRESS @_rule_@ 259 UNIT DOES NOT EXIST/OUT OF SCOPE @_rule_@ 256 REMOVED DURING SUB-SAMPLING @_rule_@ 257 UNIT ALREADY HAD A CHANCE OF SELECTION",OUTCOME CODES BETWEEN 001 AND 020 ARE FOR CATI. ALL OTHER OUTCOME CODES ARE FOR CAPI.


In [ ]:
data_df_va=data_df[data_df['name']!='FILLER']
data_df_va['valid_entry']=data_df_va['valid_entry'].str.replace(' @_rule_@', ';')
data_df_va.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,item_num,category,name,size,description,location,edited_universe,valid_entry,note
0,1,A1. HOUSEHOLD INFORMATION,HRHHID,15,HOUSEHOLD IDENTIFIER (Part 1),'1-15,ALL HHLD's IN SAMPLE,,Part 1. See Characters 71-75 for Part 2 of the Household Identifier.
1,2,A1. HOUSEHOLD INFORMATION,HRMONTH,2,MONTH OF INTERVIEW,16-17,ALL HHLDs IN SAMPLE,01 MIN VALUE; 12 MAX VALUE,
2,3,A1. HOUSEHOLD INFORMATION,HRYEAR4,4,YEAR OF INTERVIEW,18-21,ALL HHLDs IN SAMPLE,1998 MIN VALUE; 2999 MAX VALUE,
3,4,A1. HOUSEHOLD INFORMATION,HURESPLI,2,LINE NUMBER OF THE CURRENT RESPONDENT,22-23,,0 MIN VALUE; 99 MAX VALUE,
4,5,A1. HOUSEHOLD INFORMATION,HUFINAL,3,FINAL OUTCOME CODE,24-26,,"001 FULLY COMPLETE CATI INTERVIEW; 002 PARTIALLY COMPLETED CATI INTERVIEW; 003 COMPLETE BUT PERSONAL VISIT REQUESTED NEXT MONTH; 004 PARTIAL, NOT COMPLETE AT CLOSEOUT; 005 LABOR FORCE COMPLETE, SUPPLEMENT INCOMPLETE - CATI; 006 LF COMPLETE, SUPPLEMENT DK ITEMS INCOMPLETE AT CLOSEOUT–ASEC ONLY; 020 HH OCCUPIED ENTIRELY BY ARMED FORCES MEMBERS OR ALL UNDER 15 YEARS OF AGE; 201 CAPI COMPLETE; 202 CALLBACK NEEDED; 203 SUFFICIENT PARTIAL - PRECLOSEOUT; 204 SUFFICIENT PARTIAL - AT CLOSEOUT; 205 LABOR FORCE COMPLETE, - SUPPL. INCOMPLETE - CAPI; 213 LANGUAGE BARRIER; 214 UNABLE TO LOCATE; 216 NO ONE HOME; 217 TEMPORARILY ABSENT; 218 REFUSED; 219 OTHER OCCUPIED - SPECIFY; 223 ENTIRE HOUSEHOLD ARMED FORCES; 224 ENTIRE HOUSEHOLD UNDER 15; 225 TEMP. OCCUPIED W/PERSONS WITH URE; 226 VACANT REGULAR; 227 VACANT - STORAGE OF HHLD FURNITURE; 228 UNFIT, TO BE DEMOLISHED; 229 UNDER CONSTRUCTION, NOT READY; 230 CONVERTED TO TEMP BUSINESS OR STORAGE; 231 UNOCCUPIED TENT OR TRAILER SITE; 232 PERMIT GRANTED - CONSTRUCTION NOT STARTED; 233 OTHER - SPECIFY; 240 DEMOLISHED; 241 HOUSE OR TRAILER MOVED; 242 OUTSIDE SEGMENT; 243 CONVERTED TO PERM. BUSINESS OR STORAGE; 244 MERGED; 245 CONDEMNED; 247 UNUSED SERIAL NUMBER OR LISTING SHEET; 248 OTHER - SPECIFY; 258 UNLOCATABLE SAMPLE ADDRESS; 259 UNIT DOES NOT EXIST/OUT OF SCOPE; 256 REMOVED DURING SUB-SAMPLING; 257 UNIT ALREADY HAD A CHANCE OF SELECTION",OUTCOME CODES BETWEEN 001 AND 020 ARE FOR CATI. ALL OTHER OUTCOME CODES ARE FOR CAPI.


In [ ]:
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
data_df2=data_df[data_df['name']!='FILLER']
duplicateRowsDF=data_df2[data_df2.duplicated(['name'])]

In [ ]:
data_df2=data_df[data_df['name']!='FILLER']
data_df2['item_num']=range(1, len(data_df2)+1)
valid_txt = open("/content/drive/My Drive/BOC/2021validEntry.txt",'w')
for index, row in data_df2.iterrows():
  valid_txt.write('\n'+'index:'+str(index))
  valid_txt.write('\n'+'name:'+row['name'])
  valid_txt.write('\n'+'valid rules:'+'\n')
  for rule in row['valid_entry'].split('@_rule_@'):
    valid_txt.write(str(rule.strip())+'\n')
valid_txt.close()

data_df2.head(7)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,item_num,category,name,size,description,location,edited_universe,valid_entry,note
0,1,A1. HOUSEHOLD INFORMATION,HRHHID,15,HOUSEHOLD IDENTIFIER (Part 1),'1-15,ALL HHLD's IN SAMPLE,,Part 1. See Characters 71-75 for Part 2 of the Household Identifier.
1,2,A1. HOUSEHOLD INFORMATION,HRMONTH,2,MONTH OF INTERVIEW,16-17,ALL HHLDs IN SAMPLE,01 MIN VALUE @_rule_@ 12 MAX VALUE,
2,3,A1. HOUSEHOLD INFORMATION,HRYEAR4,4,YEAR OF INTERVIEW,18-21,ALL HHLDs IN SAMPLE,1998 MIN VALUE @_rule_@ 2999 MAX VALUE,
3,4,A1. HOUSEHOLD INFORMATION,HURESPLI,2,LINE NUMBER OF THE CURRENT RESPONDENT,22-23,,0 MIN VALUE @_rule_@ 99 MAX VALUE,
4,5,A1. HOUSEHOLD INFORMATION,HUFINAL,3,FINAL OUTCOME CODE,24-26,,001 FULLY COMPLETE CATI INTERVIEW @_rule_@ 002 PARTIALLY COMPLETED CATI INTERVIEW @_rule_@ 003 ...,OUTCOME CODES BETWEEN 001 AND 020 ARE FOR CATI. ALL OTHER OUTCOME CODES ARE FOR CAPI.
6,6,A1. HOUSEHOLD INFORMATION,HETENURE,2,ARE YOUR LIVING QUARTERS... (READ ANSWER CATEGORIES),29 - 30,HRINTSTA = 1 OR HUTYPB = 1-3,1 OWNED OR BEING BOUGHT BY A HH MEMBER @_rule_@ 2 RENTED FOR CASH @_rule_@ 3 OCCUPIED WITHOUT P...,NOTE: May be missing on the Basic CPS microdata files. This will be updated on later releases o...
7,7,A1. HOUSEHOLD INFORMATION,HEHOUSUT,2,TYPE OF HOUSING UNIT,31 - 32,ALL HHLDs IN SAMPLE,"0 OTHER UNIT @_rule_@ 1 HOUSE, APARTMENT, FLAT @_rule_@ 2 HU IN NONTRANSIENT HOTEL, MOTEL, ETC....",


**Create Name and Description dictionary**

In [ ]:
dict_nameDescription={}
for index, row in data_df2.iterrows():
  name = row['name']
  description = row['description']
  dict_nameDescription[name]=description

dict_nameDescription

{'GEDIV': 'DIVISION',
 'GEREG': 'REGION',
 'GESTFIPS': 'FEDERAL INFORMATION  PROCESSING STANDARDS  (FIPS) STATE CODE',
 'GTCBSA': 'SPECIFIC METROPOLITAN CBSA CODE  (SEE GEOGRAPHIC ATTACHMENT)',
 'GTCBSAST': 'PRINCIPAL CITY/BALANCE STATUS',
 'GTCBSASZ': 'Metropolitan Area (CBSA) SIZE',
 'GTCO': 'FIPS COUNTY CODE',
 'GTCSA': 'Consolidated Statistical Area (CSA) FIPS CODE  (SEE GEOGRAPHIC ATTACHMENT)',
 'GTINDVPC': 'INDIVIDUAL PRINCIPAL CITY',
 'GTMETSTA': 'METROPOLITAN STATUS',
 'HEFAMINC': 'FAMILY INCOME  (COMBINED INCOME OF ALL FAMILY MEMBERS  DURING THE LAST 12 MONTHS.  INCLUDES MONEY  FROM JOBS, NET INCOME FROM BUSINESS, FARM  OR RENT, PENSIONS, DIVIDENDS, INTEREST,  SOCIAL SECURITY PAYMENTS AND ANY OTHER  MONEY INCOME RECEIVED BY FAMILY MEMBERS  WHO ARE 15 YEARS OF AGE OR OLDER.)',
 'HEHOUSUT': 'TYPE OF HOUSING UNIT',
 'HEPHONEO': 'IS A TELEPHONE INTERVIEW ACCEPTABLE?',
 'HETELAVL': 'IS THERE A TELEPHONE ELSEWHERE ON  WHICH PEOPLE IN THIS HOUSEHOLD CAN  BE CONTACTED?',
 'HETELHHD': 

**Detailing Valid Entries**

In [ ]:
pattern_itemObject1='(^[-0-9]+.\s+(.*)'
pattern_itemRange='(^[-0-9]+.[0-9]+)\s(.*)'
#pattern_itemObject2='(^[-0-9]+.[-0-9]*)(.*)\s\s([-0-9]+.[-0-9]*)(.*)'
valid_rules={} #{name:{no.1:rule}}
for index, row in data_df2.iterrows():
  key=row['name']
  if not key in valid_rules.keys():
    valid_rules[key]={}
  for rule in row['valid_entry'].split('@_rule_@'):
    rule=rule.strip()
    if rule:
      rule0=rule.split(' ', 1)
      inner_key=rule0[0].strip()
      inner_value=rule0[1].strip()
      valid_rules[key][inner_key]=inner_value

In [ ]:
valid_rules['HUFINAL']

{'001': 'FULLY COMPLETE CATI INTERVIEW',
 '002': 'PARTIALLY COMPLETED CATI INTERVIEW',
 '003': 'COMPLETE BUT PERSONAL VISIT REQUESTED NEXT MONTH',
 '004': 'PARTIAL, NOT COMPLETE AT CLOSEOUT',
 '005': 'LABOR FORCE COMPLETE, SUPPLEMENT INCOMPLETE - CATI',
 '006': 'LF COMPLETE, SUPPLEMENT DK ITEMS INCOMPLETE AT CLOSEOUT–ASEC ONLY',
 '020': 'HH OCCUPIED ENTIRELY BY ARMED FORCES MEMBERS OR ALL UNDER 15 YEARS OF AGE',
 '201': 'CAPI COMPLETE',
 '202': 'CALLBACK NEEDED',
 '203': 'SUFFICIENT PARTIAL - PRECLOSEOUT',
 '204': 'SUFFICIENT PARTIAL - AT CLOSEOUT',
 '205': 'LABOR FORCE COMPLETE, - SUPPL. INCOMPLETE - CAPI',
 '213': 'LANGUAGE BARRIER',
 '214': 'UNABLE TO LOCATE',
 '216': 'NO ONE HOME',
 '217': 'TEMPORARILY ABSENT',
 '218': 'REFUSED',
 '219': 'OTHER OCCUPIED - SPECIFY',
 '223': 'ENTIRE HOUSEHOLD ARMED FORCES',
 '224': 'ENTIRE HOUSEHOLD UNDER 15',
 '225': 'TEMP. OCCUPIED W/PERSONS WITH URE',
 '226': 'VACANT REGULAR',
 '227': 'VACANT - STORAGE OF HHLD FURNITURE',
 '228': 'UNFIT, TO BE DEM

In [ ]:
#dict(zip(range(1, 4), range(1, 4)))
va_test0=valid_rules
#va_test0={'PEHRUSLT':{'-4': 'VARIES', '0': 'MIN VALUE', '198': 'MAX VALUE'}}
import itertools
import copy
from collections import OrderedDict
va_test1=copy.deepcopy(va_test0)

va_test2={}
pattern_va_range='([0-9]+)\W([0-9]+)'
pattern_va_key='([-0-9]+)\W'
for name, item in va_test0.items():
  valueRange_min=None
  valueRange_max=None
  valueRange_text=''
  for inner_key, inner_value in item.items():
    if inner_value.strip().startswith('MIN VALUE') or inner_value.strip().startswith('MAX VALUE'):
      valueRange_text=valueRange_text+str(inner_key)+": "+str(inner_value)+'; '
      if inner_value.strip().startswith('MIN VALUE'):
        valueRange_min=int(inner_key)
      else:
        valueRange_max=int(inner_key)
      del va_test1[name][inner_key]
      if valueRange_min != None and valueRange_max != None:
        va_test1[name].update(dict(zip(range(valueRange_min,valueRange_max+1), [valueRange_text]*len(range(valueRange_min,valueRange_max+1)))))
    elif re.match(pattern_va_range, inner_key):
      matchObj=re.match(pattern_va_range, inner_key)
      keyRnage_min=int(matchObj.group(1).strip())
      keyRnage_max=int(matchObj.group(2).strip())
      del va_test1[name][inner_key]
      va_test1[name].update(dict(zip(range(keyRnage_min,keyRnage_max+1), [str(inner_value)]*len(range(keyRnage_min,keyRnage_max+1)))))

for name, item in va_test1.items():
  va_test2[name]={}
  for inner_key, inner_value in item.items():
    try:
      va_test2[name][int(inner_key)]=inner_value
    except:
      if re.match(pattern_va_key, inner_key):
        matchObj=re.match(pattern_va_key, inner_key)
        inner_key=int(matchObj.group(1).strip())
        va_test2[name][inner_key]=inner_value

va_rules_withoutEmpty={}        
for key, value in va_test2.items():
  if value:
    va_rules_withoutEmpty[key]=value

#va_rules_withoutEmpty
#dict(itertools.islice(va_rules_withoutEmpty.items(),5 ))
#va_rules_withoutEmpty['HUFINAL']

In [ ]:
#va_test2['PTERNHLY']

In [ ]:
va_df_dataList=[]
for name, item in va_rules_withoutEmpty.items():
  for rule_num, rule_description in item.items():
    va_df_dataList.append([name, rule_num, rule_description])
va_df=pd.DataFrame(va_df_dataList)
va_df.columns=['name', 'rule_num', 'rule_description']
va_df.head()

,name,rule_num,rule_description
0,HRMONTH,1,01: MIN VALUE; 12: MAX VALUE;
1,HRMONTH,2,01: MIN VALUE; 12: MAX VALUE;
2,HRMONTH,3,01: MIN VALUE; 12: MAX VALUE;
3,HRMONTH,4,01: MIN VALUE; 12: MAX VALUE;
4,HRMONTH,5,01: MIN VALUE; 12: MAX VALUE;


dump dictionary for paring larger dataset

In [ ]:
pickle.dump(data_df2, open('/content/drive/My Drive/BOC/DataReference/data_df2','wb'))
pickle.dump(va_df, open('/content/drive/My Drive/BOC/DataReference/va_df','wb'))
pickle.dump(data_df_va, open('/content/drive/My Drive/BOC/DataReference/data_df_va','wb'))
pickle.dump(dict_nameDescription, open('/content/drive/My Drive/BOC/DataReference/dict_nameDescription','wb'))
pickle.dump(va_test2, open('/content/drive/My Drive/BOC/DataReference/va_test2','wb'))


**Parse CPS CSV**

In [ ]:
from collections import defaultdict
non_reference_dict=defaultdict(list)

In [ ]:
df_CPS0=pd.read_csv('/content/drive/My Drive/BOC/may21pub_sample1000.csv')
df_CPS0.insert(0,'No.', range(1, len(df_CPS0)+1))
df_CPS0.insert(0,'date', '202105')
df_CPS0.head()

,date,No.,hrhhid2,HUFINAL,OCCURNUM,HUINTTYP,HURESPLI,HUPRSCNT,HUTYPEA,HUTYPB,HUTYPC,HUBUS,HUBUSL1,HUBUSL2,HUBUSL3,HUBUSL4,HRMIS,HRMONTH,HRYEAR4,HRLONGLK,qstnum,gereg,gestfips,gediv,hehousut,hxhousut,hephoneo,hxphoneo,hetelavl,hxtelavl,hetelhhd,hxtelhhd,hrhtype,hrintsta,hrnumhou,hefaminc,hxfaminc,hwhhwgt,hwhhwtln,PULINENO,PUCHINHH,PUWK,PUBUS1,PUDIS,PULAY,PUHROFF1,PUHROFF2,PUHROT1,PUHROT2,PUABSOT,PUBUSCK1,PUBUSCK2,PUBUSCK3,PUBUSCK4,PURETOT,PUHRCK1,PUHRCK2,PUHRCK3,PUHRCK4,PUHRCK5,PUHRCK6,PUHRCK7,PUHRCK12,PULAYDT,PULAY6M,PULAYAVR,PULK,PULKAVR,PULAYCK1,PULAYCK2,PULAYCK3,PUDWCK1,PUDWCK2,PUDWCK3,PUDWCK4,PUDWCK5,PUJHCK1,PUJHCK2,PUJHDP1O,PUJHCK3,PUJHCK4,PUJHCK5,PULKM2,PULKM3,PULKM4,PULKM5,PULKM6,PULKDK1,PULKDK2,PULKDK3,PULKDK4,PULKDK5,PULKDK6,PULKPS1,PULKPS2,PULKPS3,PULKPS4,PULKPS5,PULKPS6,PUIOCK1,PUIOCK2,PUIOCK3,PUIODP1,PUIODP2,PUIODP3,PUIO1MFG,PUIO2MFG,ptern2,pternh1c,PUNLFCK1,PUNLFCK2,PUSLFPRX,PUDIS1,PUDIS2,PUBUS2OT,perrp,pxrrp,pxage,peafnow,pxafnow,pesex,pxsex,pemaritl,pxmaritl,pxrace1,pehspnon,pxhspnon,peeduca,pxeduca,peafever,pxafever,peafwhn1,pxafwhn1,peafwhn2,peafwhn3,peafwhn4,pespouse,pxspouse,penatvty,pxnatvty,pemntvty,pxmntvty,pefntvty,pxfntvty,pxinusyr,pedipged,pxdipged,pehgcomp,pxhgcomp,pecyc,pxcyc,pepar1,pxpar1,pepar2,pxpar2,pepar1typ,pxpar1typ,pepar2typ,pxpar2typ,prdasian,prmarsta,ptdtrace,prdthsp,prpertyp,prfamnum,prfamtyp,prfamrel,prnmchld,prchld,prcitflg,prcitshp,prinuyer,prtage,prtfage,pecohab,pxcohab,peabspdo,peabsrsn,pedwavl,pedwavr,pedwlko,pedwlkwk,pedwrsn,pedwwk,pedwwnto,pedw4wk,pehractt,pehract1,pehract2,pehravl,pehrftpt,pehrrsn1,pehrrsn2,pehrrsn3,pehruslt,pehrusl1,pehrusl2,pehrwant,pejhrsn,pejhwant,pejhwko,pelayavl,pelaydur,pelayfto,pelaylk,pelkavl,pelkdur,pelkfto,pelkll1o,pelkll2o,pelklwo,pelkm1,pemjnum,pemjot,pemlr,penlfact,penlfjh,penlfret,peret1,pxabspdo,pxabsrsn,pxdwavl,pxdwavr,pxdwlko,pxdwlkwk,pxdwrsn,pxdwwk,pxdwwnto,pxdw4wk,pxhractt,pxhract1,pxhract2,pxhravl,pxhrftpt,pxhrrsn1,pxhrrsn2,pxhrrsn3,pxhruslt,pxhrusl1,pxhrusl2,pxhrwant,pxjhrsn,pxjhwant,pxjhwko,pxlayavl,pxlaydur,pxlayfto,pxlaylk,pxlkavl,pxlkdur,pxlkfto,pxlkll1o,pxlkll2o,pxlklwo,pxlkm1,pxmjnum,pxmjot,pxmlr,pxnlfact,pxnlfjh,pxnlfret,pxret1,prabsrea,prcivlf,prdisc,premphrs,prempnot,prexplf,prftlf,prhrusl,prjobsea,prpthrs,prptrea,prunedur,pruntype,prwksch,prwkstat,prwntjob,peio1icd,peio2icd,ptio1ocd,ptio2ocd,peio1cow,peio2cow,pepdemp1,pepdemp2,pxio1icd,pxio2icd,pxio1ocd,pxio2ocd,pxio1cow,pxio2cow,pxpdemp1,pxpdemp2,pxnmemp1,pxnmemp2,prioelg,premp,prcow1,prcow2,prnagws,prnagpws,prdtcow1,prdtcow2,prmjind1,prmjind2,primind1,primind2,prmjocc1,prmjocc2,prdtind1,prdtind2,prdtocc1,prdtocc2,pragna,prsjmj,prcowpg,prmjocgr,peernper,peernhry,peernuot,peernwkp,peernrt,pternh2,pternh1o,peernhro,ptern,peernlab,peerncov,pxernper,pxernhry,pxernuot,pxernwkp,pxernrt,pxernh2,pxernh1o,pxernhro,pxern,pxernlab,pxerncov,prerelg,pternwa,pternhly,prwernal,prhernal,peschlvl,peschenr,peschft,pxschlvl,pxschenr,pxschft,prnlfsch,pedisear,pediseye,pedisrem,pedisphy,pedisdrs,pedisout,pxdisear,pxdiseye,pxdisrem,pxdisphy,pxdisdrs,pxdisout,prdisflg,pecert1,pecert2,pecert3,pxcert1,pxcert2,pxcert3,pwsswgt,pwlgwgt,pwvetwgt,pworwgt,pwfmwgt,pwcmpwgt,pthr,ptwk,ptot,ptnmemp1,ptnmemp2,hrhhid,gtcbsa,gtco,gtcbsast,gtcbsasz,gtcsa,gtmetsta,gtindvpc
0,202105,1,12011,201,1,2,1,0,-1,-1,-1,2,-1,-1,-1,-1,3,5,2021,2,1,3,1,6,1,0,1,0,-1,1,1,0,3,1,3,11,22,17871696.0,1.0,1.0,9.0,1.0,-1.0,-1.0,-1.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,5.0,5.0,-1.0,2.0,3.0,5.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,3.0,3.0,1.0,2.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,40.0,0.0,0.0,2.0,0.0,1.0,0.0,6.0,0.0,0.0,2.0,0.0,39.0,0.0,2.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,57.0,-1.0,57.0,-1.0,57.0,-1.0,-1.0,1.0,0.0,-1.0,1.0,-1.0,1.0,-1,1,-1,1,-1,1,-1,1,-1.0,7.0,1.0,-1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,19.0,0,-1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1

In [ ]:
colname0=[i.upper() for i in df_CPS0.columns.tolist() if i not in ['date', 'No.']]
colnameDesc=[None]*(len(colname0)*2)
count=0
for colname in colname0:
  if colname in dict_nameDescription.keys():
    description = dict_nameDescription[colname]
  else:
    description = 'no reference for Such terminology In CPS'
    if colname!='DATE' and colname!='NO.':
      description='None'
      if [description,'description','no reference for Such terminology In CPS'] not in non_reference_dict[colname]:
        non_reference_dict[colname].append([description,'description','no reference for Such terminology In CPS'])
        print(f'no reference in CPS for {colname}') 
  colnameDesc[count]=description
  count+=1
  colnameDesc[count]=description
  count+=1
df_CPS3=copy.deepcopy(df_CPS0)
df_CPS3.columns=[i.upper() for i in df_CPS0.columns.tolist()]
df_CPS3.head()

no reference in CPS for PRINUYER
no reference in CPS for PRSJMJ


,DATE,NO.,HRHHID2,HUFINAL,OCCURNUM,HUINTTYP,HURESPLI,HUPRSCNT,HUTYPEA,HUTYPB,HUTYPC,HUBUS,HUBUSL1,HUBUSL2,HUBUSL3,HUBUSL4,HRMIS,HRMONTH,HRYEAR4,HRLONGLK,QSTNUM,GEREG,GESTFIPS,GEDIV,HEHOUSUT,HXHOUSUT,HEPHONEO,HXPHONEO,HETELAVL,HXTELAVL,HETELHHD,HXTELHHD,HRHTYPE,HRINTSTA,HRNUMHOU,HEFAMINC,HXFAMINC,HWHHWGT,HWHHWTLN,PULINENO,PUCHINHH,PUWK,PUBUS1,PUDIS,PULAY,PUHROFF1,PUHROFF2,PUHROT1,PUHROT2,PUABSOT,PUBUSCK1,PUBUSCK2,PUBUSCK3,PUBUSCK4,PURETOT,PUHRCK1,PUHRCK2,PUHRCK3,PUHRCK4,PUHRCK5,PUHRCK6,PUHRCK7,PUHRCK12,PULAYDT,PULAY6M,PULAYAVR,PULK,PULKAVR,PULAYCK1,PULAYCK2,PULAYCK3,PUDWCK1,PUDWCK2,PUDWCK3,PUDWCK4,PUDWCK5,PUJHCK1,PUJHCK2,PUJHDP1O,PUJHCK3,PUJHCK4,PUJHCK5,PULKM2,PULKM3,PULKM4,PULKM5,PULKM6,PULKDK1,PULKDK2,PULKDK3,PULKDK4,PULKDK5,PULKDK6,PULKPS1,PULKPS2,PULKPS3,PULKPS4,PULKPS5,PULKPS6,PUIOCK1,PUIOCK2,PUIOCK3,PUIODP1,PUIODP2,PUIODP3,PUIO1MFG,PUIO2MFG,PTERN2,PTERNH1C,PUNLFCK1,PUNLFCK2,PUSLFPRX,PUDIS1,PUDIS2,PUBUS2OT,PERRP,PXRRP,PXAGE,PEAFNOW,PXAFNOW,PESEX,PXSEX,PEMARITL,PXMARITL,PXRACE1,PEHSPNON,PXHSPNON,PEEDUCA,PXEDUCA,PEAFEVER,PXAFEVER,PEAFWHN1,PXAFWHN1,PEAFWHN2,PEAFWHN3,PEAFWHN4,PESPOUSE,PXSPOUSE,PENATVTY,PXNATVTY,PEMNTVTY,PXMNTVTY,PEFNTVTY,PXFNTVTY,PXINUSYR,PEDIPGED,PXDIPGED,PEHGCOMP,PXHGCOMP,PECYC,PXCYC,PEPAR1,PXPAR1,PEPAR2,PXPAR2,PEPAR1TYP,PXPAR1TYP,PEPAR2TYP,PXPAR2TYP,PRDASIAN,PRMARSTA,PTDTRACE,PRDTHSP,PRPERTYP,PRFAMNUM,PRFAMTYP,PRFAMREL,PRNMCHLD,PRCHLD,PRCITFLG,PRCITSHP,PRINUYER,PRTAGE,PRTFAGE,PECOHAB,PXCOHAB,PEABSPDO,PEABSRSN,PEDWAVL,PEDWAVR,PEDWLKO,PEDWLKWK,PEDWRSN,PEDWWK,PEDWWNTO,PEDW4WK,PEHRACTT,PEHRACT1,PEHRACT2,PEHRAVL,PEHRFTPT,PEHRRSN1,PEHRRSN2,PEHRRSN3,PEHRUSLT,PEHRUSL1,PEHRUSL2,PEHRWANT,PEJHRSN,PEJHWANT,PEJHWKO,PELAYAVL,PELAYDUR,PELAYFTO,PELAYLK,PELKAVL,PELKDUR,PELKFTO,PELKLL1O,PELKLL2O,PELKLWO,PELKM1,PEMJNUM,PEMJOT,PEMLR,PENLFACT,PENLFJH,PENLFRET,PERET1,PXABSPDO,PXABSRSN,PXDWAVL,PXDWAVR,PXDWLKO,PXDWLKWK,PXDWRSN,PXDWWK,PXDWWNTO,PXDW4WK,PXHRACTT,PXHRACT1,PXHRACT2,PXHRAVL,PXHRFTPT,PXHRRSN1,PXHRRSN2,PXHRRSN3,PXHRUSLT,PXHRUSL1,PXHRUSL2,PXHRWANT,PXJHRSN,PXJHWANT,PXJHWKO,PXLAYAVL,PXLAYDUR,PXLAYFTO,PXLAYLK,PXLKAVL,PXLKDUR,PXLKFTO,PXLKLL1O,PXLKLL2O,PXLKLWO,PXLKM1,PXMJNUM,PXMJOT,PXMLR,PXNLFACT,PXNLFJH,PXNLFRET,PXRET1,PRABSREA,PRCIVLF,PRDISC,PREMPHRS,PREMPNOT,PREXPLF,PRFTLF,PRHRUSL,PRJOBSEA,PRPTHRS,PRPTREA,PRUNEDUR,PRUNTYPE,PRWKSCH,PRWKSTAT,PRWNTJOB,PEIO1ICD,PEIO2ICD,PTIO1OCD,PTIO2OCD,PEIO1COW,PEIO2COW,PEPDEMP1,PEPDEMP2,PXIO1ICD,PXIO2ICD,PXIO1OCD,PXIO2OCD,PXIO1COW,PXIO2COW,PXPDEMP1,PXPDEMP2,PXNMEMP1,PXNMEMP2,PRIOELG,PREMP,PRCOW1,PRCOW2,PRNAGWS,PRNAGPWS,PRDTCOW1,PRDTCOW2,PRMJIND1,PRMJIND2,PRIMIND1,PRIMIND2,PRMJOCC1,PRMJOCC2,PRDTIND1,PRDTIND2,PRDTOCC1,PRDTOCC2,PRAGNA,PRSJMJ,PRCOWPG,PRMJOCGR,PEERNPER,PEERNHRY,PEERNUOT,PEERNWKP,PEERNRT,PTERNH2,PTERNH1O,PEERNHRO,PTERN,PEERNLAB,PEERNCOV,PXERNPER,PXERNHRY,PXERNUOT,PXERNWKP,PXERNRT,PXERNH2,PXERNH1O,PXERNHRO,PXERN,PXERNLAB,PXERNCOV,PRERELG,PTERNWA,PTERNHLY,PRWERNAL,PRHERNAL,PESCHLVL,PESCHENR,PESCHFT,PXSCHLVL,PXSCHENR,PXSCHFT,PRNLFSCH,PEDISEAR,PEDISEYE,PEDISREM,PEDISPHY,PEDISDRS,PEDISOUT,PXDISEAR,PXDISEYE,PXDISREM,PXDISPHY,PXDISDRS,PXDISOUT,PRDISFLG,PECERT1,PECERT2,PECERT3,PXCERT1,PXCERT2,PXCERT3,PWSSWGT,PWLGWGT,PWVETWGT,PWORWGT,PWFMWGT,PWCMPWGT,PTHR,PTWK,PTOT,PTNMEMP1,PTNMEMP2,HRHHID,GTCBSA,GTCO,GTCBSAST,GTCBSASZ,GTCSA,GTMETSTA,GTINDVPC
0,202105,1,12011,201,1,2,1,0,-1,-1,-1,2,-1,-1,-1,-1,3,5,2021,2,1,3,1,6,1,0,1,0,-1,1,1,0,3,1,3,11,22,17871696.0,1.0,1.0,9.0,1.0,-1.0,-1.0,-1.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,5.0,5.0,-1.0,2.0,3.0,5.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,3.0,3.0,1.0,2.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,40.0,0.0,0.0,2.0,0.0,1.0,0.0,6.0,0.0,0.0,2.0,0.0,39.0,0.0,2.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,57.0,-1.0,57.0,-1.0,57.0,-1.0,-1.0,1.0,0.0,-1.0,1.0,-1.0,1.0,-1,1,-1,1,-1,1,-1,1,-1.0,7.0,1.0,-1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,19.0,0,-1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1

In [ ]:
num=0
for key, item in non_reference_dict.items():
  if num<=10:
    print(key, item)
    num+=1

PRINUYER [['None', 'description', 'no reference for Such terminology In CPS']]
PRSJMJ [['None', 'description', 'no reference for Such terminology In CPS']]


In [ ]:
for i in colname0:
  colName=df_CPS3.columns.tolist()
  df_CPS3.insert(colName.index(i)+1,'PARSE_'+str(i),None)
df_CPS3.head()

,DATE,NO.,HRHHID2,PARSE_HRHHID2,HUFINAL,PARSE_HUFINAL,OCCURNUM,PARSE_OCCURNUM,HUINTTYP,PARSE_HUINTTYP,HURESPLI,PARSE_HURESPLI,HUPRSCNT,PARSE_HUPRSCNT,HUTYPEA,PARSE_HUTYPEA,HUTYPB,PARSE_HUTYPB,HUTYPC,PARSE_HUTYPC,HUBUS,PARSE_HUBUS,HUBUSL1,PARSE_HUBUSL1,HUBUSL2,PARSE_HUBUSL2,HUBUSL3,PARSE_HUBUSL3,HUBUSL4,PARSE_HUBUSL4,HRMIS,PARSE_HRMIS,HRMONTH,PARSE_HRMONTH,HRYEAR4,PARSE_HRYEAR4,HRLONGLK,PARSE_HRLONGLK,QSTNUM,PARSE_QSTNUM,GEREG,PARSE_GEREG,GESTFIPS,PARSE_GESTFIPS,GEDIV,PARSE_GEDIV,HEHOUSUT,PARSE_HEHOUSUT,HXHOUSUT,PARSE_HXHOUSUT,HEPHONEO,PARSE_HEPHONEO,HXPHONEO,PARSE_HXPHONEO,HETELAVL,PARSE_HETELAVL,HXTELAVL,PARSE_HXTELAVL,HETELHHD,PARSE_HETELHHD,HXTELHHD,PARSE_HXTELHHD,HRHTYPE,PARSE_HRHTYPE,HRINTSTA,PARSE_HRINTSTA,HRNUMHOU,PARSE_HRNUMHOU,HEFAMINC,PARSE_HEFAMINC,HXFAMINC,PARSE_HXFAMINC,HWHHWGT,PARSE_HWHHWGT,HWHHWTLN,PARSE_HWHHWTLN,PULINENO,PARSE_PULINENO,PUCHINHH,PARSE_PUCHINHH,PUWK,PARSE_PUWK,PUBUS1,PARSE_PUBUS1,PUDIS,PARSE_PUDIS,PULAY,PARSE_PULAY,PUHROFF1,PARSE_PUHROFF1,PUHROFF2,PARSE_PUHROFF2,PUHROT1,PARSE_PUHROT1,PUHROT2,PARSE_PUHROT2,PUABSOT,PARSE_PUABSOT,PUBUSCK1,PARSE_PUBUSCK1,PUBUSCK2,PARSE_PUBUSCK2,PUBUSCK3,PARSE_PUBUSCK3,PUBUSCK4,PARSE_PUBUSCK4,PURETOT,PARSE_PURETOT,PUHRCK1,PARSE_PUHRCK1,PUHRCK2,PARSE_PUHRCK2,PUHRCK3,PARSE_PUHRCK3,PUHRCK4,PARSE_PUHRCK4,PUHRCK5,PARSE_PUHRCK5,PUHRCK6,PARSE_PUHRCK6,PUHRCK7,PARSE_PUHRCK7,PUHRCK12,PARSE_PUHRCK12,PULAYDT,PARSE_PULAYDT,PULAY6M,PARSE_PULAY6M,PULAYAVR,PARSE_PULAYAVR,PULK,PARSE_PULK,PULKAVR,PARSE_PULKAVR,PULAYCK1,PARSE_PULAYCK1,PULAYCK2,PARSE_PULAYCK2,PULAYCK3,PARSE_PULAYCK3,PUDWCK1,PARSE_PUDWCK1,PUDWCK2,PARSE_PUDWCK2,PUDWCK3,PARSE_PUDWCK3,PUDWCK4,PARSE_PUDWCK4,PUDWCK5,PARSE_PUDWCK5,PUJHCK1,PARSE_PUJHCK1,PUJHCK2,PARSE_PUJHCK2,PUJHDP1O,PARSE_PUJHDP1O,PUJHCK3,PARSE_PUJHCK3,PUJHCK4,PARSE_PUJHCK4,PUJHCK5,PARSE_PUJHCK5,PULKM2,PARSE_PULKM2,PULKM3,PARSE_PULKM3,PULKM4,PARSE_PULKM4,PULKM5,PARSE_PULKM5,PULKM6,PARSE_PULKM6,PULKDK1,PARSE_PULKDK1,PULKDK2,PARSE_PULKDK2,PULKDK3,PARSE_PULKDK3,PULKDK4,PARSE_PULKDK4,PULKDK5,PARSE_PULKDK5,PULKDK6,PARSE_PULKDK6,PULKPS1,PARSE_PULKPS1,PULKPS2,PARSE_PULKPS2,PULKPS3,PARSE_PULKPS3,PULKPS4,PARSE_PULKPS4,PULKPS5,PARSE_PULKPS5,PULKPS6,PARSE_PULKPS6,PUIOCK1,PARSE_PUIOCK1,PUIOCK2,PARSE_PUIOCK2,PUIOCK3,PARSE_PUIOCK3,PUIODP1,PARSE_PUIODP1,PUIODP2,PARSE_PUIODP2,PUIODP3,PARSE_PUIODP3,PUIO1MFG,PARSE_PUIO1MFG,PUIO2MFG,PARSE_PUIO2MFG,PTERN2,PARSE_PTERN2,PTERNH1C,PARSE_PTERNH1C,PUNLFCK1,PARSE_PUNLFCK1,PUNLFCK2,PARSE_PUNLFCK2,PUSLFPRX,PARSE_PUSLFPRX,PUDIS1,PARSE_PUDIS1,PUDIS2,PARSE_PUDIS2,PUBUS2OT,PARSE_PUBUS2OT,PERRP,PARSE_PERRP,PXRRP,PARSE_PXRRP,PXAGE,PARSE_PXAGE,PEAFNOW,PARSE_PEAFNOW,PXAFNOW,PARSE_PXAFNOW,PESEX,PARSE_PESEX,PXSEX,PARSE_PXSEX,PEMARITL,PARSE_PEMARITL,PXMARITL,PARSE_PXMARITL,PXRACE1,PARSE_PXRACE1,PEHSPNON,PARSE_PEHSPNON,PXHSPNON,PARSE_PXHSPNON,PEEDUCA,PARSE_PEEDUCA,PXEDUCA,PARSE_PXEDUCA,PEAFEVER,PARSE_PEAFEVER,PXAFEVER,PARSE_PXAFEVER,PEAFWHN1,PARSE_PEAFWHN1,PXAFWHN1,PARSE_PXAFWHN1,PEAFWHN2,PARSE_PEAFWHN2,PEAFWHN3,PARSE_PEAFWHN3,PEAFWHN4,PARSE_PEAFWHN4,PESPOUSE,PARSE_PESPOUSE,PXSPOUSE,PARSE_PXSPOUSE,PENATVTY,PARSE_PENATVTY,PXNATVTY,PARSE_PXNATVTY,PEMNTVTY,PARSE_PEMNTVTY,PXMNTVTY,PARSE_PXMNTVTY,PEFNTVTY,PARSE_PEFNTVTY,PXFNTVTY,PARSE_PXFNTVTY,PXINUSYR,PARSE_PXINUSYR,PEDIPGED,PARSE_PEDIPGED,PXDIPGED,PARSE_PXDIPGED,PEHGCOMP,PARSE_PEHGCOMP,PXHGCOMP,PARSE_PXHGCOMP,PECYC,PARSE_PECYC,PXCYC,PARSE_PXCYC,PEPAR1,PARSE_PEPAR1,PXPAR1,PARSE_PXPAR1,PEPAR2,PARSE_PEPAR2,PXPAR2,PARSE_PXPAR2,PEPAR1TYP,PARSE_PEPAR1TYP,PXPAR1TYP,PARSE_PXPAR1TYP,PEPAR2TYP,PARSE_PEPAR2TYP,PXPAR2TYP,PARSE_PXPAR2TYP,PRDASIAN,PARSE_PRDASIAN,PRMARSTA,PARSE_PRMARSTA,PTDTRACE,PARSE_PTDTRACE,PRDTHSP,PARSE_PRDTHSP,PRPERTYP,PARSE_PRPERTYP,PRFAMNUM,PARSE_PRFAMNUM,PRFAMTYP,PARSE_PRFAMTYP,PRFAMREL,PARSE_PRFAMREL,PRNMCHLD,PARSE_PRNMCHLD,PRCHLD,PARSE_PRCHLD,PRCITFLG,PARSE_PRCITFLG,PRCITSHP,PARSE_PRCITSHP,PRINUYER,PARSE_PRINUYER,PRTAGE,PARSE_PRTAGE,PRTFAGE,PARSE_PRTFAGE,PECOHAB,PARSE_PECOHAB,PXCOHAB,PARSE_PXCOHAB,PEABSPDO,PARSE_PEABSPDO,PEABSRSN,PARSE_PEABSRSN,PEDWAVL,PARSE_PEDWAVL,PEDWAVR,PARSE_PEDWAVR,PEDW

In [ ]:
#df_CPS3_sample=df_CPS3.iloc[0:5]
colname2=df_CPS3.columns.to_list()
name_reference_problem_list=[]
for i in range(len(df_CPS3)):
  for j in range(len(colname2)):
    if (j+1)%2==0:
      name=colname2[j-1]
      rule_num=df_CPS3.iloc[i,j-1]
      if name in va_test2.keys():
        if not va_test2[name].values():
          df_CPS3.iloc[i,j]='empty rules'
          description=data_df2[data_df2['name']==name]['description'].values[0]
          if [description,'all entries','empty rules about all entries'] not in non_reference_dict[name]:
            non_reference_dict[name].append([description,'all entries','empty rules about all entries'])
        else:
          if rule_num in va_test2[name].keys():
            parse_content=va_test2[name][int(rule_num)]
            if re.match('^=(.*)', parse_content.strip()):
              matchObj=re.match('^=(.*)', parse_content.strip())
              parse_content=matchObj.group(1).strip()
            df_CPS3.iloc[i,j]=parse_content
          else:
            df_CPS3.iloc[i,j]='no reference for such rule number'
            description=data_df2[data_df2['name']==name]['description'].values[0]
            if [description, str(rule_num),'no reference for such rule number'] not in non_reference_dict[name]:
              non_reference_dict[name].append([description, str(rule_num),'no reference for such rule number'])
      else:
        if not name in name_reference_problem_list:
          name_reference_problem_list.append(name)

In [ ]:
num=0
for key, item in non_reference_dict.items():
  if num<=10:
    print(key, item)
    num+=1

PRINUYER [['None', 'description', 'no reference for Such terminology In CPS']]
PRSJMJ [['None', 'description', 'no reference for Such terminology In CPS']]
HRHHID2 [['HOUSEHOLD IDENTIFIER (part 2)', 'all entries', 'empty rules about all entries']]
OCCURNUM [['Unique person identifier.  Valid only within  any specific month.', 'all entries', 'empty rules about all entries']]
HUPRSCNT [['NUMBER OF ACTUAL AND  ATTEMPTED PERSONAL CONTACTS', '0', 'no reference for such rule number']]
HUTYPEA [['TYPE A NONINTERVIEW REASON', '-1', 'no reference for such rule number']]
HUTYPB [['TYPE B NON-INTERVIEW REASON', '-1', 'no reference for such rule number']]
HUTYPC [['TYPE C NON-INTERVIEW REASON', '-1', 'no reference for such rule number']]
HUBUSL1 [['ENTER LINE NUMBER  FOR HUBUS = 1', '-1', 'no reference for such rule number']]
HUBUSL2 [['See  BUSL1', '-1', 'no reference for such rule number']]
HUBUSL3 [['See BUSL1', '-1', 'no reference for such rule number']]


In [ ]:
#add description in first row
colnameDesc.insert(0,'DATE')
colnameDesc.insert(1,'NO.')
df_CPS3.loc[-1] = None  # adding a row
df_CPS3.index = df_CPS3.index + 1  # shifting index
df_CPS3 = df_CPS3.sort_index()  # sorting by index
df_CPS3.loc[0]=colnameDesc
df_CPS3.iloc[174:176]['PRDASIAN']


174    2
175   -1
Name: PRDASIAN, dtype: object

In [ ]:
#df_CPS3.iloc[174:175]['PARSE_PRDASIAN']=str('Chinese')

In [ ]:

df_CPS3.to_csv('/content/drive/My Drive/BOC/parse_may21pub_sample1000.csv', encoding='utf-8-sig', index=False)

**Detailing Edited Universe**

In [ ]:

ed_txt = open("/content/drive/My Drive/BOC/2021EditedUniverse.txt",'w')
for index, row in data_df2.iterrows():
  ed_txt.write('\n'+'index:'+str(index))
  ed_txt.write('\n'+'name:'+row['name'])
  ed_txt.write('\n'+'edited_universe rules:'+'\n')
  ed_txt.write(row['edited_universe']+'\n')
ed_txt.close()


In [ ]:
import pdb
edited_rules={} #{name:{inner_name:1,2,3-8}}
reference_keys=data_df2['name'].tolist()

#key, row = ('HEPHONEO', 'HETELHHD = 1 OR HETELAVL = 1')
#ed_list=row.split()

for index, row in data_df2.iterrows():
  key=row['name']
  ed_list=row['edited_universe'].split(' ')
  #print(ed_list)
  index_list=[]
  name_list=[]
  if not key in edited_rules.keys():
    edited_rules[key]={}
  for i in range(len(ed_list)):
    name_list_fake=[i for i in re.findall('[aA-zZ]+[0-9]*', ed_list[i].strip()) if i]
    #pdb.set_trace()
    for name in name_list_fake:
      if name and name in reference_keys:
        index_list.append(i)
        name_list.append(name)
  index_list.append(len(ed_list))
  #print(name_list)
  #print(index_list)
  ed_dictionary={}
  for j in range(len(index_list)-1):
      value_list=[]  
      for k in range(index_list[j]+1, index_list[j+1]):
        value_list_sub=[]
        value_list_sub=[int(i) for i in re.findall('^-[0-9]+|[0-9]+', ed_list[k].strip()) if i]
        value_list.extend(value_list_sub) 
      #print(value_list)   
      if not name_list[j] in ed_dictionary.keys():
          ed_dictionary[name_list[j]]=value_list
      else:
          value_list_add=[int(value) for value in value_list if value not in ed_dictionary[name_list[j]]]
          value_list=ed_dictionary[name_list[j]]
          value_list.extend(value_list_add)
          ed_dictionary[name_list[j]]=value_list
  #print(ed_dictionary)
  edited_rules[key]=ed_dictionary

edited_rules_withoutEmpty={}
edited_rules_withoutEmpty2={}
for key, value in edited_rules.items():
  if value:
    edited_rules_withoutEmpty[key]=value
    if key!='PRDISFLG':
      edited_rules_withoutEmpty2[key]=value
#edited_rules_withoutEmpty['PRNLFSCH']

edited_rules_withoutEmpty2

{'HEPHONEO': {'HETELAVL': [1], 'HETELHHD': [1]},
 'HETELAVL': {'HETELHHD': [2]},
 'HETELHHD': {'HRINTSTA': [1]},
 'HETENURE': {'HRINTSTA': [1], 'HUTYPB': [1, 3]},
 'HWHHWGT': {'HRINTSTA': [1]},
 'HWHHWTLN': {'HRINTSTA': [1]},
 'PEABSPDO': {'PEABSRSN': [4, 12, 14]},
 'PEABSRSN': {'PEMLR': [2]},
 'PEAFEVER': {'PRTAGE': [17]},
 'PEAFNOW': {'PRPERTYP': [2, 3]},
 'PEAFWHN1': {'PEAFEVER': [1]},
 'PEAFWHN2': {'PEAFEVER': [1]},
 'PEAFWHN3': {'PEAFEVER': [1]},
 'PEAFWHN4': {'PEAFEVER': [1]},
 'PECERT1': {'PRPERTYP': [2]},
 'PECERT2': {'PECERT1': [1]},
 'PECERT3': {'PECERT2': [1]},
 'PECYC': {'PEEDUCA': [40, 42]},
 'PEDISDRS': {'PRPERTYP': [2]},
 'PEDISEAR': {'PRPERTYP': [2]},
 'PEDISEYE': {'PRPERTYP': [2]},
 'PEDISOUT': {'PRPERTYP': [2]},
 'PEDISPHY': {'PRPERTYP': [2]},
 'PEDISREM': {'PRPERTYP': [2]},
 'PEDW4WK': {'PEDWWK': [1]},
 'PEDWAVL': {'PEDWLKWK': [1], 'PEDWWK': [2]},
 'PEDWAVR': {'PEDWAVL': [2]},
 'PEDWLKO': {'PEDWRSN': [1, 11], 'PUDWCK4': [1, 3]},
 'PEDWRSN': {'PUDWCK4': [4, -1]},
 'PE

In [ ]:
ed_df_dataList=[]
for name, item in edited_rules_withoutEmpty2.items():
  for reference_key, reference_nums in item.items():
    ed_df_dataList.append([name, reference_key, reference_nums])
ed_df=pd.DataFrame(ed_df_dataList)
ed_df.columns=['name', 'reference_key', 'reference_nums']
ed_df

,name,reference_key,reference_nums
0,HETENURE,HRINTSTA,[1]
1,HETENURE,HUTYPB,"[1, 3]"
2,HETELHHD,HRINTSTA,[1]
3,HETELAVL,HETELHHD,[2]
4,HEPHONEO,HETELHHD,[1]
5,HEPHONEO,HETELAVL,[1]
6,HWHHWGT,HRINTSTA,[1]
7,HWHHWTLN,HRINTSTA,[1]
8,PERRP,PRPERTYP,"[1, 2, 3]"
9,PRTAGE,PRPERTYP,"[1, 2, 0, 3]"


In [ ]:

ed_df_dataList=[]
for index, row in ed_df.iterrows():
  name=row['name']
  reference_key=row['reference_key']
  reference_rules=''
  reference_nums=row['reference_nums']
  for num in reference_nums:
    if num in va_test2[reference_key].keys():
      reference_rules=reference_rules+str(num)+':'+va_test2[reference_key][num]+';'
    else:
      reference_rules='no reference when num='+str(num)+';'
  full_rules=data_df_va[data_df_va['name']==reference_key]['valid_entry'].item()
  ed_df_dataList.append([name, reference_key, reference_nums, reference_rules, full_rules])

ed_df2=pd.DataFrame(ed_df_dataList)
ed_df2.columns=['name', 'reference_key', 'reference_nums', 'reference_rules', 'full_rules']
ed_df2.head()

,name,reference_key,reference_nums,reference_rules,full_rules
0,HETENURE,HRINTSTA,[1],1:INTERVIEW;,1 INTERVIEW; 2 TYPE A NON-INTERVIEW; 3 TYPE B NON-INTERVIEW; 4 TYPE C NON-INTERVIEW
1,HETENURE,HUTYPB,"[1, 3]",1:VACANT REGULAR;3:VACANT-STORAGE OF HHLD FURNITURE;,1 VACANT REGULAR; 2 TEMPORARILY OCCUPIED BY PERSONS W/ URE; 3 VACANT-STORAGE OF HHLD FURNITURE;...
2,HETELHHD,HRINTSTA,[1],1:INTERVIEW;,1 INTERVIEW; 2 TYPE A NON-INTERVIEW; 3 TYPE B NON-INTERVIEW; 4 TYPE C NON-INTERVIEW
3,HETELAVL,HETELHHD,[2],2:NO;,1 YES; 2 NO
4,HEPHONEO,HETELHHD,[1],1:YES;,1 YES; 2 NO


**Creating tables for Gelphi**


In [ ]:
#data_df2 + ed_df2-->df_nodes
#ed_df2-->df_edges
df_edges_dataList=[]
for index, row in ed_df2.iterrows():
  Source_name=row['name']
  Target_name=row['reference_key']
  Source = int(data_df2[data_df2['name']==row['name']]['item_num'].item())-1
  Target = int(data_df2[data_df2['name']==row['reference_key']]['item_num'].item())-1
  Weight = len(row['reference_nums'])
  df_edges_dataList.append([Source, Target, Weight,Source_name, Target_name])

df_edges=pd.DataFrame(df_edges_dataList)
df_edges.columns=['Source','Target', 'Weight', 'Source_name', 'Targe_name']
df_edges.to_csv('/content/drive/My Drive/BOC/edges.csv', encoding='utf-8-sig')
df_edges.head()

,Source,Target,Weight,Source_name,Targe_name
0,5,15,1,HETENURE,HRINTSTA
1,5,12,2,HETENURE,HUTYPB
2,7,15,1,HETELHHD,HRINTSTA
3,8,7,1,HETELAVL,HETELHHD
4,9,7,1,HEPHONEO,HETELHHD


In [ ]:
df_nodes_dataList=[]
ed_df2_nameList=ed_df2['name'].tolist()
for index, row in data_df2.iterrows():
  id=int(row['item_num'])-1
  label=row['name']
  category=row['category']
  description=row['description']
  note=row['note']
  #full_rules0=row['valid_entry']
  #full_rules=full_rules0.replace(' @_rule_@',';')
  full_rules=va_test0[label]
  reference_rules='no edited universe related valid rules'
  reference_keys='no edited universe related keys'
  if label in ed_df2_nameList:
    for colname, content in ed_df2[ed_df2['name']==label]['reference_rules'].items():
      reference_rules=content
    for colname, content in ed_df2[ed_df2['name']==label]['reference_key'].items():
      reference_keys=content
  df_nodes_dataList.append([id, label, category, description, note, reference_keys, reference_rules, full_rules])
df_nodes=pd.DataFrame(df_nodes_dataList)
df_nodes.columns=['id', 'label', 'category', 'description', 'note', 'ed_reference_keys', 'ed_reference_rules', 'full_valid_entry_rules']
df_nodes.head(7)

,id,label,category,description,note,ed_reference_keys,ed_reference_rules,full_valid_entry_rules
0,0,HRHHID,A1. HOUSEHOLD INFORMATION,HOUSEHOLD IDENTIFIER (Part 1),Part 1. See Characters 71-75 for Part 2 of the Household Identifier.,no edited universe related keys,no edited universe related valid rules,{}
1,1,HRMONTH,A1. HOUSEHOLD INFORMATION,MONTH OF INTERVIEW,,no edited universe related keys,no edited universe related valid rules,"{'01': 'MIN VALUE', '12': 'MAX VALUE'}"
2,2,HRYEAR4,A1. HOUSEHOLD INFORMATION,YEAR OF INTERVIEW,,no edited universe related keys,no edited universe related valid rules,"{'1998': 'MIN VALUE', '2999': 'MAX VALUE'}"
3,3,HURESPLI,A1. HOUSEHOLD INFORMATION,LINE NUMBER OF THE CURRENT RESPONDENT,,no edited universe related keys,no edited universe related valid rules,"{'0': 'MIN VALUE', '99': 'MAX VALUE'}"
4,4,HUFINAL,A1. HOUSEHOLD INFORMATION,FINAL OUTCOME CODE,OUTCOME CODES BETWEEN 001 AND 020 ARE FOR CATI. ALL OTHER OUTCOME CODES ARE FOR CAPI.,no edited universe related keys,no edited universe related valid rules,"{'001': 'FULLY COMPLETE CATI INTERVIEW', '002': 'PARTIALLY COMPLETED CATI INTERVIEW', '003': 'CO..."
5,5,HETENURE,A1. HOUSEHOLD INFORMATION,ARE YOUR LIVING QUARTERS... (READ ANSWER CATEGORIES),NOTE: May be missing on the Basic CPS microdata files. This will be updated on later releases o...,HUTYPB,1:VACANT REGULAR;3:VACANT-STORAGE OF HHLD FURNITURE;,"{'1': 'OWNED OR BEING BOUGHT BY A HH MEMBER', '2': 'RENTED FOR CASH', '3': 'OCCUPIED WITHOUT PAY..."
6,6,HEHOUSUT,A1. HOUSEHOLD INFORMATION,TYPE OF HOUSING UNIT,,no edited universe related keys,no edited universe related valid rules,"{'0': 'OTHER UNIT', '1': 'HOUSE, APARTMENT, FLAT', '2': 'HU IN NONTRANSIENT HOTEL, MOTEL, ETC.',..."


In [ ]:
df_nodes.to_csv('/content/drive/My Drive/BOC/nodes.csv', encoding='utf-8-sig', index=False)

**For Testing CPS3**

In [ ]:
# testing logic: for most records: exclude blank and -1, if 'no reference' appears....It may be abnormal.

In [ ]:
test_possible_problem_list=[]
for i in range(2,len(df_CPS3)):
  for j in range(len(colname2)):
    if (j+1)%2!=0:
      name=colname2[j]
      rule_num=df_CPS3.iloc[i,j]
      try:
        rule_num=int(rule_num) #exclude blank cell: NaN
        if rule_num != int(-1) and df_CPS3.iloc[i,j+1] == 'no reference for such rule number':
          if not name in test_possible_problem_list:
            test_possible_problem_list.append(name)
      except:
        pass

        
print(test_possible_problem_list)
print(len(test_possible_problem_list))


['HUPRSCNT', 'HETELAVL', 'PUIO1MFG', 'PEERNRT', 'PUWK', 'PULAY', 'PUABSOT', 'PULK', 'PUHROT2', 'PUHROFF1']
10


In [ ]:
end_time=time.time()
total_time=round((end_time-start_time)/60,2)
print(f'total time is {total_time} min')

total time is 5.79 min


In [ ]:
num=0
for key, item in non_reference_dict.items():
  if num<=35:
    print(key, item)
    num+=1

PRINUYER [['None', 'description', 'no reference for Such terminology In CPS']]
PRSJMJ [['None', 'description', 'no reference for Such terminology In CPS']]
HRHHID2 [['HOUSEHOLD IDENTIFIER (part 2)', 'all entries', 'empty rules about all entries']]
OCCURNUM [['Unique person identifier.  Valid only within  any specific month.', 'all entries', 'empty rules about all entries']]
HUPRSCNT [['NUMBER OF ACTUAL AND  ATTEMPTED PERSONAL CONTACTS', '0', 'no reference for such rule number']]
HUTYPEA [['TYPE A NONINTERVIEW REASON', '-1', 'no reference for such rule number']]
HUTYPB [['TYPE B NON-INTERVIEW REASON', '-1', 'no reference for such rule number']]
HUTYPC [['TYPE C NON-INTERVIEW REASON', '-1', 'no reference for such rule number']]
HUBUSL1 [['ENTER LINE NUMBER  FOR HUBUS = 1', '-1', 'no reference for such rule number']]
HUBUSL2 [['See  BUSL1', '-1', 'no reference for such rule number']]
HUBUSL3 [['See BUSL1', '-1', 'no reference for such rule number']]
HUBUSL4 [['See BUSL1', '-1', 'no refere

In [ ]:
len(non_reference_dict)

361

In [ ]:
dataList=[]
num=0
for terminology, items in non_reference_dict.items():
  for item in items:
    description,entry, problem=item
    num+=1
    dataList.append(['202105',num,terminology,description, entry, problem])
non_reference_df=pd.DataFrame(dataList)
non_reference_df.columns=['Date','NO.','Terminology','Description','Entry','Problem']

In [ ]:
non_reference_df.tail(5)

,Date,NO.,Terminology,Description,Entry,Problem
561,202105,562,PEDISDRS,DOES … HAVE DIFFICULTY DRESSING OR BATHING?,-1,no reference for such rule number
562,202105,563,PEDISOUT,"BECAUSE OF A PHYSICAL, MENTAL, OR EMOTIONAL CONDITION DOES…HAVE DIFFICULTY DOING ERRANDS ALONE...",-1,no reference for such rule number
563,202105,564,PRDISFLG,DOES THIS PERSON HAVE ANY OF THESE DISABILITY CONDITIONS?,-1,no reference for such rule number
564,202105,565,PECERT1,DOES … HAVE A CURRENTLY ACTIVE PROFESSIONAL CERTIFICATION OR A STATE OR INDUSTRY LICENSE? DO ...,-1,no reference for such rule number
565,202105,566,HUINTTYP,TYPE OF INTERVIEW,-1,no reference for such rule number


In [ ]:
len(non_reference_df)

566